In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

/media/samuele/Disco/PycharmProjectsUbuntu/MARL-CER


In [2]:
import jax
import jax.numpy as jnp
from flax import nnx
from functools import partial
from jax_tqdm import scan_tqdm

from algorithms.utils import save_state_multiagent

import numpy as np

# jax.config.update("jax_debug_nans", True)

2025-03-08 19:05:49.093946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741457149.108841  240906 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741457149.113349  240906 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from ernestogym.envs_jax.multi_agent.env import RECEnv

In [4]:
def my_env_creator(params, battery_type):
    env = RECEnv(params, battery_type)
    return env

In [5]:
from ernestogym.envs_jax.multi_agent.utils import parameter_generator

In [6]:
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_train_split_demands.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_train.yaml"
# world = "ernestogym/envs_jax/single_agent/world_fading.yaml"

# battery_type = 'fading'
# battery_type = 'degrading'
battery_type = 'degrading_dropflow'

params = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params

{'num_battery_agents': 3,
 'num_passive_houses': 0,
 'batteries': [{'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 288.0,
    'temp_ambient': 298.15,
    'nominal_cost': 3000.0},
   'bounds': {'voltage': {'low': 288.0, 'high': 398.4},
    'current': {'low': -180.0, 'high': 60.0},
    'power': {'low': -71712.0, 'high': 23904.0},
    'temperature': {'low': 273.15, 'high': 323.15},
    'temp_ambient': {'low': 273.15, 'high': 323.15},
    'soc': {'low': 0.2, 'high': 1.0},
    'soh': {'low': 0.6, 'high': 1.0}},
   'init': {'voltage': 310.8,
    'current': 0.0,
    'power': 0.0,
    'temperature': 293.15,
    'temp_ambient': 293.15,
    'soc': 0.2,
    'soh': 1.0}},
  {'sign_convention': 'passive',
   'params': {'nominal_voltage': 350.4,
    'nominal_capacity': 60.0,
    'nominal_dod': 0.8,
    'nominal_lifetime': 3000,
    'v_max': 398.4,
    'v_min': 2

In [7]:
pack_options = "ernestogym/ernesto_jax/data/battery_new/pack.yaml"
ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_pack.yaml"
# ecm = "ernestogym/ernesto_jax/data/battery_new/models/electrical/thevenin_fading_pack.yaml"
r2c = "ernestogym/ernesto_jax/data/battery_new/models/thermal/r2c_thermal_pack.yaml"
# bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
bolun = "ernestogym/ernesto_jax/data/battery_new/models/aging/bolun_pack.yaml"
world = "ernestogym/envs_jax/multi_agent/world_deg_test.yaml"
# world = "ernestogym/envs_jax/multi_agent/world_deg_test_split_demands.yaml"
# world = "ernestogym/envs_jax/single_agent/world_fading.yaml"

params_testing = parameter_generator(
    input_var='current',
    battery_options=pack_options,
    electrical_model=ecm,
    thermal_model=r2c,
    aging_model=bolun,
    world_options=world,
    use_reward_normalization=True

)

params_testing['step']

3600

In [8]:
from algorithms.multi_agent_ppo import make_train, train        #, train_for, train_for_flax
from flax.core.frozen_dict import freeze, unfreeze

In [9]:
num_envs = 4

config = {
    "LR_BATTERIES": 1e-3,
    "LR_REC": 1e-3,
    "LR_BATTERIES_MIN": 5e-5,
    "LR_REC_MIN": 5e-5,
    "NUM_ENVS": num_envs,
    "NUM_STEPS": 8192, #2048 #10000,
    # "TOTAL_TIMESTEPS": 100000 * num_envs,
    "TOTAL_TIMESTEPS": 876000 // 2 * num_envs,
    "UPDATE_EPOCHS": 10,
    "NUM_MINIBATCHES": 64, #32,
    "NORMALIZE_ENV": False,
    "GAMMA": 0.99,
    "GAE_LAMBDA": 0.95,
    "CLIP_EPS": 0.2,
    "ENT_COEF": 0.01,
    "VF_COEF": 0.5,
    "MAX_GRAD_NORM": 0.5,
    "NET_ARCH": (64, 32, 16), #(16, 16, 16),
    "LSTM_NET_ARCH": (24, 12),
    "ACTIVATION": 'tanh',
    "LSTM_ACTIVATION": 'tanh',
    "LOGISTIC_FUNCTION_TO_ACTOR": False,
    "ENV_NAME": None,
    "LR_SCHEDULE": 'cosine',
    'DEBUG': False,
    "NORMALIZE_ADVANTAGES": True,
    "ENV_TYPE": 'normal',
    'NETWORK_TYPE_BATTERIES': 'recurrent_actor_critic',
    'NETWORK_TYPE_REC': 'recurrent_actor_critic',
    'USE_WEIGHT_DECAY': True,
    'NORMALIZE_NN_INPUTS': False
}

env_type = config['ENV_TYPE']

rng = jax.random.PRNGKey(42)

env = my_env_creator(params, battery_type)
env, train_state = make_train(config, env)

env_testing = my_env_creator(params_testing, battery_type)

val_rng = jax.random.PRNGKey(51)
val_num_iters = 8670*5

# out = train_jit(rng)

config = freeze(config)

In [10]:
config

FrozenDict({
    LR_BATTERIES: 0.001,
    LR_REC: 0.001,
    LR_BATTERIES_MIN: 5e-05,
    LR_REC_MIN: 5e-05,
    NUM_ENVS: 4,
    NUM_STEPS: 8192,
    TOTAL_TIMESTEPS: 1752000,
    UPDATE_EPOCHS: 10,
    NUM_MINIBATCHES: 64,
    NORMALIZE_ENV: False,
    GAMMA: 0.99,
    GAE_LAMBDA: 0.95,
    CLIP_EPS: 0.2,
    ENT_COEF: 0.01,
    VF_COEF: 0.5,
    MAX_GRAD_NORM: 0.5,
    NET_ARCH: (64, 32, 16),
    LSTM_NET_ARCH: (24, 12),
    ACTIVATION: 'tanh',
    LSTM_ACTIVATION: 'tanh',
    LOGISTIC_FUNCTION_TO_ACTOR: False,
    ENV_NAME: None,
    LR_SCHEDULE: 'cosine',
    DEBUG: False,
    NORMALIZE_ADVANTAGES: True,
    ENV_TYPE: 'normal',
    NETWORK_TYPE_BATTERIES: 'recurrent_actor_critic',
    NETWORK_TYPE_REC: 'recurrent_actor_critic',
    USE_WEIGHT_DECAY: True,
    NORMALIZE_NN_INPUTS: False,
    NUM_UPDATES: 53,
    MINIBATCH_SIZE: 512,
    BATTERY_ACTION_SPACE_SIZE: 1,
    BATTERY_OBSERVATION_SPACE_SIZE: 11,
    BATTERY_NUM_SEQUENCES: 7,
    REC_ACTION_SPACE_SIZE: 3,
    REC_OBS_KEYS:

In [11]:
env.battery_agents

['battery_agent_0', 'battery_agent_1', 'battery_agent_2']

In [12]:
env.valorization_incentive_coeff

8e-06

In [13]:
networks_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)

In [14]:
networks_batteries

StackedRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x738b1c335580>,
    dot_general=<function dot_general at 0x738bd7166200>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(3, 16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x738b1c334fe0>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x738bd6e1ce00>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x738b1c15a020>,
    dot_general=<function dot_general at 0x738bd7166200>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x738b1c1598a0>,
    out_features=32

In [15]:
network_rec

RECRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(64,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x738b1073eca0>,
    dot_general=<function dot_general at 0x738bd7166200>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x738b1073df80>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x738bd6e1ce00>>, Linear(
    bias=Param(
      value=Array(shape=(32,), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x738b1073f240>,
    dot_general=<function dot_general at 0x738bd7166200>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x738b1073f9c0>,
    out_features=32,
    param_dt

In [ ]:
import time
rng = jax.random.PRNGKey(42)
t0 = time.time()

# with jax.checking_leaks():
out = train(env, config, train_state, rng, validate=True, freq_val=10, val_env=env_testing, val_rng=val_rng, val_num_iters=val_num_iters)

train_state = out['runner_state'].train_state

print(f"time: {time.time() - t0:.2f} s")

Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

Running for 53 iterations:   0%|          | 0/53 [00:00<?, ?it/s]

Running for 43,350 iterations:   0%|          | 0/43350 [00:00<?, ?it/s]

curr_iter: 0
	r_tot: -8488.84375
	r_trad: -3324.380859375
	r_deg: -3253.27099609375
	r_clip: -2076.6796875
	r_glob: 165.48712158203125
	r_rec: 1365781.5


In [17]:
jax.tree.map(lambda x: x.shape, out['metrics'])

{'actions': {'REC_agent': (53, 8192, 4, 3),
  'battery_agent_0': (53, 8192, 4, 1),
  'battery_agent_1': (53, 8192, 4, 1),
  'battery_agent_2': (53, 8192, 4, 1)},
 'balance_minus': (53, 8192, 4),
 'balance_plus': (53, 8192, 4),
 'buy_prices': (53, 8192, 4, 3),
 'demands': (53, 8192, 4, 3),
 'energy_to_batteries': (53, 8192, 4, 3),
 'generations': (53, 8192, 4, 3),
 'norm_reward': {'r_clipping': (53, 8192, 4, 3),
  'r_deg': (53, 8192, 4, 3),
  'r_op': (53, 8192, 4, 3),
  'r_trad': (53, 8192, 4, 3)},
 'pure_reward': {'r_clipping': (53, 8192, 4, 3),
  'r_deg': (53, 8192, 4, 3),
  'r_op': (53, 8192, 4, 3),
  'r_trad': (53, 8192, 4, 3)},
 'r_glob': (53, 8192, 4, 3),
 'r_tot': (53, 8192, 4, 3),
 'rec_reward': (53, 8192, 4),
 'self_consumption': (53, 8192, 4),
 'sell_prices': (53, 8192, 4, 3),
 'soc': (53, 8192, 4, 3),
 'soh': (53, 8192, 4, 3),
 'tot_incentives': (53, 8192, 4),
 'weig_reward': {'r_clipping': (53, 8192, 4, 3),
  'r_deg': (53, 8192, 4, 3),
  'r_op': (53, 8192, 4, 3),
  'r_trad':

In [18]:
jax.tree.map(lambda x: jnp.isnan(x).any(), out['metrics'])

{'actions': {'REC_agent': Array(False, dtype=bool),
  'battery_agent_0': Array(False, dtype=bool),
  'battery_agent_1': Array(False, dtype=bool),
  'battery_agent_2': Array(False, dtype=bool)},
 'balance_minus': Array(False, dtype=bool),
 'balance_plus': Array(False, dtype=bool),
 'buy_prices': Array(False, dtype=bool),
 'demands': Array(False, dtype=bool),
 'energy_to_batteries': Array(False, dtype=bool),
 'generations': Array(False, dtype=bool),
 'norm_reward': {'r_clipping': Array(False, dtype=bool),
  'r_deg': Array(False, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(False, dtype=bool)},
 'pure_reward': {'r_clipping': Array(False, dtype=bool),
  'r_deg': Array(False, dtype=bool),
  'r_op': Array(False, dtype=bool),
  'r_trad': Array(False, dtype=bool)},
 'r_glob': Array(False, dtype=bool),
 'r_tot': Array(False, dtype=bool),
 'rec_reward': Array(False, dtype=bool),
 'self_consumption': Array(False, dtype=bool),
 'sell_prices': Array(False, dtype=bool),
 'soc':

In [19]:
jax.tree.map(lambda x: jnp.isnan(x).any(axis=jnp.arange(1, x.ndim)), out['metrics'])

{'actions': {'REC_agent': Array([False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False],      dtype=bool),
  'battery_agent_0': Array([False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False],      dtype=bool),
  'battery_agent_1': Array([False, False, False, False, False, False,

In [20]:
out['metrics']['actions']['REC_agent'][0]

Array([[[0.5453448 , 0.09716875, 0.35748652],
        [0.5193089 , 0.10173705, 0.37895402],
        [0.01253856, 0.00490008, 0.9825614 ],
        [0.18243557, 0.21223632, 0.6053282 ]],

       [[0.71765864, 0.210902  , 0.07143932],
        [0.01893652, 0.01598809, 0.9650754 ],
        [0.6243214 , 0.35083884, 0.02483975],
        [0.1853863 , 0.14233556, 0.6722782 ]],

       [[0.06938731, 0.21868265, 0.7119299 ],
        [0.20072512, 0.5244259 , 0.27484894],
        [0.1695939 , 0.8234505 , 0.00695556],
        [0.05780643, 0.8471019 , 0.09509163]],

       ...,

       [[0.02096681, 0.13884793, 0.8401853 ],
        [0.18446824, 0.03055218, 0.7849796 ],
        [0.45908564, 0.49417672, 0.04673774],
        [0.5931211 , 0.01579966, 0.39107925]],

       [[0.16292714, 0.76805955, 0.06901339],
        [0.3037581 , 0.03505822, 0.6611837 ],
        [0.8019946 , 0.12799732, 0.07000804],
        [0.29204872, 0.2511831 , 0.45676813]],

       [[0.54288816, 0.28903124, 0.16808057],
        [0.

In [21]:
out['metrics']['actions']['REC_agent'][1]

Array([[[1.9691026e-02, 9.3937987e-01, 4.0929053e-02],
        [2.8539810e-01, 5.1618713e-01, 1.9841485e-01],
        [2.0574641e-01, 6.8712586e-01, 1.0712783e-01],
        [9.6500665e-01, 2.2573546e-02, 1.2419767e-02]],

       [[2.1319851e-01, 4.1304637e-02, 7.4549687e-01],
        [2.0027424e-01, 1.4193909e-01, 6.5778667e-01],
        [6.4417887e-01, 9.1499731e-02, 2.6432145e-01],
        [4.8644727e-01, 7.4448168e-02, 4.3910453e-01]],

       [[3.6425468e-01, 3.7414607e-01, 2.6159918e-01],
        [6.0867310e-02, 1.7875864e-01, 7.6037407e-01],
        [3.7395751e-01, 2.3026319e-01, 3.9577934e-01],
        [3.4797889e-01, 1.3127598e-01, 5.2074510e-01]],

       ...,

       [[2.8353397e-04, 4.2316234e-01, 5.7655412e-01],
        [5.0618285e-01, 4.7905910e-01, 1.4758099e-02],
        [5.2037996e-01, 1.0861581e-01, 3.7100431e-01],
        [3.6573764e-02, 1.8258754e-02, 9.4516748e-01]],

       [[9.2043781e-01, 6.9139533e-02, 1.0422648e-02],
        [7.4746937e-02, 7.6945603e-01, 1.557

In [22]:
np.isnan(out['metrics']['actions']['REC_agent'][1]).any()

False

In [23]:
jax.tree.map(lambda x: np.nonzero(np.isnan(x)), out['metrics'])

{'actions': {'REC_agent': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_0': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_1': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64)),
  'battery_agent_2': (array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64),
   array([], dtype=int64))},
 'balance_minus': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'balance_plus': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'buy_prices': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'demands': (array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64),
  array([], dtype=int64)),
 'energy_to_batteries': (array([], dtype=int64),
  array

In [24]:
out['metrics']['actions']['REC_agent']

Array([[[[5.45344830e-01, 9.71687511e-02, 3.57486516e-01],
         [5.19308925e-01, 1.01737052e-01, 3.78954023e-01],
         [1.25385579e-02, 4.90007503e-03, 9.82561409e-01],
         [1.82435572e-01, 2.12236315e-01, 6.05328202e-01]],

        [[7.17658639e-01, 2.10902005e-01, 7.14393184e-02],
         [1.89365186e-02, 1.59880891e-02, 9.65075374e-01],
         [6.24321401e-01, 3.50838840e-01, 2.48397514e-02],
         [1.85386300e-01, 1.42335564e-01, 6.72278225e-01]],

        [[6.93873093e-02, 2.18682647e-01, 7.11929917e-01],
         [2.00725123e-01, 5.24425924e-01, 2.74848938e-01],
         [1.69593900e-01, 8.23450506e-01, 6.95556449e-03],
         [5.78064322e-02, 8.47101927e-01, 9.50916335e-02]],

        ...,

        [[2.09668111e-02, 1.38847932e-01, 8.40185285e-01],
         [1.84468240e-01, 3.05521842e-02, 7.84979582e-01],
         [4.59085643e-01, 4.94176716e-01, 4.67377380e-02],
         [5.93121111e-01, 1.57996621e-02, 3.91079247e-01]],

        [[1.62927136e-01, 7.680595

In [25]:
out['metrics']['pure_reward']['r_clipping'][3]

Array([[[   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ]],

       [[ -63.85321  ,    0.       ,    0.       ],
        [   0.       , -120.48578  ,    0.       ],
        [   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ]],

       [[-235.96245  ,    0.       ,    0.       ],
        [   0.       , -167.03685  ,   -3.2210844],
        [   0.       ,    0.       ,    0.       ],
        [ -11.463507 ,    0.       ,    0.       ]],

       ...,

       [[   0.       ,    0.       , -407.64545  ],
        [   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ],
        [-520.4514   ,    0.       ,    0.       ]],

       [[   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0.       ],
        [   0.       ,    0.       ,    0. 

In [26]:
network_batteries, optimizer_batteries, network_rec, optimizer_rec = nnx.merge(train_state.graph_def, train_state.state)
network_batteries

StackedRecurrentActorCritic(
  act_layers=[Linear(
    bias=Param(
      value=Array(shape=(3, 64), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7970d875d4e0>,
    dot_general=<function dot_general at 0x7971a11fa160>,
    dtype=None,
    in_features=16,
    kernel=Param(
      value=Array(shape=(3, 16, 64), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7970d875cf40>,
    out_features=64,
    param_dtype=<class 'jax.numpy.float32'>,
    precision=None,
    use_bias=True
  ), <PjitFunction of <function tanh at 0x7971a0ee8d60>>, Linear(
    bias=Param(
      value=Array(shape=(3, 32), dtype=float32)
    ),
    bias_init=<function constant.<locals>.init at 0x7970d84e9f80>,
    dot_general=<function dot_general at 0x7971a11fa160>,
    dtype=None,
    in_features=64,
    kernel=Param(
      value=Array(shape=(3, 64, 32), dtype=float32)
    ),
    kernel_init=<function orthogonal.<locals>.init at 0x7970d84e9800>,
    out_features=32

## State saving

In [27]:
from algorithms.utils import save_state_multiagent
save_state_multiagent(network_batteries, network_rec, config, params, train_info=out['metrics'], val_info=out.get('val_info'), env_type=env_type, additional_info='fairness_coeff_'+str(params['fairness_coeff']))

In [28]:
out

{'metrics': {'actions': {'REC_agent': Array([[[[5.45344830e-01, 9.71687511e-02, 3.57486516e-01],
            [5.19308925e-01, 1.01737052e-01, 3.78954023e-01],
            [1.25385579e-02, 4.90007503e-03, 9.82561409e-01],
            [1.82435572e-01, 2.12236315e-01, 6.05328202e-01]],
   
           [[7.17658639e-01, 2.10902005e-01, 7.14393184e-02],
            [1.89365186e-02, 1.59880891e-02, 9.65075374e-01],
            [6.24321401e-01, 3.50838840e-01, 2.48397514e-02],
            [1.85386300e-01, 1.42335564e-01, 6.72278225e-01]],
   
           [[6.93873093e-02, 2.18682647e-01, 7.11929917e-01],
            [2.00725123e-01, 5.24425924e-01, 2.74848938e-01],
            [1.69593900e-01, 8.23450506e-01, 6.95556449e-03],
            [5.78064322e-02, 8.47101927e-01, 9.50916335e-02]],
   
           ...,
   
           [[2.09668111e-02, 1.38847932e-01, 8.40185285e-01],
            [1.84468240e-01, 3.05521842e-02, 7.84979582e-01],
            [4.59085643e-01, 4.94176716e-01, 4.67377380e-02],
